<a href="https://colab.research.google.com/github/trentgooden/DataAnalyticsClub/blob/main/DAC_Python_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data Analytics Club Python Series - Number 3

One of the more common tasks where Python is useful is to download files from the web, either directly or via an Application Programming Interface.  We will look at both.  But first, since we are using Colab, we need to understand where to store files (Google Drive) and how to reference them.

In [ ]:
# import the drive module
from google.colab import drive
# mount your Google Drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [ ]:
import requests

# Define the URL of the file to download
file_url = "https://thedocs.worldbank.org/en/doc/5d903e848db1d1b83e0ec8f744e55570-0350012021/related/CMO-Historical-Data-Monthly.xlsx"

# Define the local file path to save the downloaded file
local_file_path = "/content/drive/MyDrive/CMO-Historical-Data-Monthly.xlsx"

# Download the file from the URL
response = requests.get(file_url)

#200 is a successful request in HTTP, 404 not found, 403 forbidden, 500 server error
if response.status_code == 200:
    # Save the downloaded file to Google Drive
    # 'wb' means write binary, which is typically used to write non text files like spreadsheets
    with open(local_file_path, 'wb') as file:
        file.write(response.content)
    print("File downloaded and saved to Google Drive successfully!")
else:
    print(f"Failed to download the file. Status code: {response.status_code}")


File downloaded and saved to Google Drive successfully!


In [ ]:
import pandas as pd
from google.colab import drive

#Mount Google Drive
#drive.mount('/content/drive/MyDrive')

local_file_path = "/content/drive/MyDrive/CMO-Historical-Data-Monthly.xlsx"

#df = pd.read_excel(local_file_path)
df = pd.read_excel(local_file_path, sheet_name = 'Monthly Prices', skiprows=4)

print(df.head())

df1 = df.melt(id_vars=["Commodity"], var_name="Date",  value_name="Value")

print(df1)

#Application Programming Interfaces (APIs)

We will use the US Census Bureaus API to query the Components of Population Change for a few counties, then use

In [ ]:
#Must request API Key
#User Guide: https://www.census.gov/data/developers/guidance/api-user-guide.html
#Available APIs: https://www.census.gov/data/developers/data-sets.html



import requests  #for web request
import pandas as pd
%matplotlib inline

#with open('C:/users/tgooden/Desktop/census_api_key.txt') as key:
#    api_key=key.read().strip()

#print(api_key)

#HOST = "https://api.census.gov/data"
year = "2018"
dsource = "pep"
dname = "components"
cols = "GEONAME,NATURALINC,DOMESTICMIG,INTERNATIONALMIG"
state = "13"
county = "121,067,089,117,135"
api_key = "<your api key here>"

#when using an "f" (formatted string) in front of a string,
#all the variables inside curly brackets are read and evaluated at runtime
base_url = f'https://api.census.gov/data/{year}/{dsource}/{dname}'
data_url = f'{base_url}?get={cols}&for=county:{county}&in=state:{state}&key={api_key}'
print("The data url is: " + data_url)
response=requests.get(data_url)
print(response.text)

data=response.json()
df=pd.DataFrame(data[1:], columns=data[0]).\
    rename(columns={"NATURALINC": "Natural Increase", "DOMESTICMIG": "Net Domestic Mig", "INTERNATIONALMIG":"Net Foreign Mig"})
df['fips']=df.state+df.county
df.set_index('fips',inplace=True)
df.drop(columns=['state','county'])
df=df.astype(dtype={'Natural Increase':'int64','Net Domestic Mig':'int64','Net Foreign Mig':'int64'})
df

labels=df['GEONAME'].str.split(' ',expand=True)[0]
ax=df.plot.bar(rot=0, title='Components of Population Change 2017-18')
ax.set_xticklabels(labels)
ax.set_xlabel('')

Use the Series `map()` method.
E.g. To filter by the length of a column values:

After executing the cell above, a new file named 'Sample file.txt' will appear in your [drive.google.com](https://drive.google.com/) file list.